In [16]:
!pip install -q auto-ts pystan~=2.14 fbprophet

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      Cython>=0.22 and NumPy are required.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [57 lines of output]
      /tmp/pip-install-7qj9on0w/fbprophet_5f1faba46b214a6f8c43076ab91f0ce5/setup.py:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import (
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/fbprophet
      creating build/lib/fbprophet/stan_model
      Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enf

In [4]:
import sys
#give the absolute path of the repo folder here
sys.path.insert(1, '/home/ediss4/sarosh/personal/hackathon/ediss_hackathon_team7_2024')

In [7]:
from src.read import load_data_xlsx, read_and_print_file


In [12]:
track_data_path = '/home/ediss4/sarosh/personal/hackathon/ediss_hackathon_team7_2024/data/Feature_Track.xlsx'
simulated_data_path = '/home/ediss4/sarosh/personal/hackathon/ediss_hackathon_team7_2024/data/Feature_Simulation.xlsx'
track_data = load_data_xlsx(track_data_path)
simulated_data = load_data_xlsx(track_data_path)

In [7]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        # parametreleri
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        # batch-layer
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class FTTransformer(keras.Model):

    def __init__(self, 
            categories,
            num_continuous,
            dim,
            dim_out,
            depth,
            embedding_dim,
            heads,
            attn_dropout,
            ff_dropout,
            mlp_hidden,
            normalize_continuous = True):

        super(FTTransformer, self).__init__()

        # --> continuous inputs
        self.embedding_dim = embedding_dim
        self.normalize_continuous = normalize_continuous
        if normalize_continuous:
            self.continuous_normalization = layers.LayerNormalization()

        # --> categorical inputs

        # embedding
        self.embedding_layers = []
        for number_of_classes in categories:
            self.embedding_layers.append(layers.Embedding(input_dim = number_of_classes, output_dim = dim))
        # self.embedding_layers_cont = []
        # for number_of_classes in range(177):
        #     self.embedding_layers_cont.append(layers.Embedding(input_dim = 10, output_dim = dim))
        self.embb = layers.Embedding(input_dim = 1000, output_dim = dim)
        self.flatten_output = layers.Flatten()

        # concatenation
        self.embedded_concatenation = layers.Concatenate(axis=1)
        self.cont_embedded_concatenation = layers.Concatenate(axis=1)

        # adding transformers
        self.transformers = []
        for _ in range(depth):
            self.transformers.append(TransformerBlock(dim, heads, dim))
        self.flatten_transformer_output = layers.Flatten()

        # --> MLP
        self.pre_concatenation = layers.Concatenate(axis=1)

        # mlp layers
        self.mlp_layers = []
        for size, activation in mlp_hidden:
            self.mlp_layers.append(layers.Dense(size, activation=activation))

        self.output_layer = layers.Dense(dim_out)

    def call(self, inputs):
        continuous_inputs  = inputs[0]
        categorical_inputs = inputs[1:]
        # print(continuous_inputs.shape, type(categorical_inputs))
        # --> continuous
        if self.normalize_continuous:
            continuous_inputs = self.continuous_normalization(continuous_inputs)
    
        # cont_embedding_outputs = []
        # print(continuous_inputs.transpoghjse()[0].shape)
        # for continuous_input, embedding_layer in zip(tf.unstack(tf.transpose(continuous_inputs)), self.embedding_layers_cont):
        #     cont_embedding_outputs.append(tf.expand_dims(embedding_layer(continuous_input),axis=1))
        # continuous_inputs = self.cont_embedded_concatenation(cont_embedding_outputs)
        continuous_inputs = self.embb(continuous_inputs)
        # print(cont_embedding_outputs[0].shape)
        # --> categorical
        embedding_outputs = []
        for categorical_input, embedding_layer in zip(categorical_inputs, self.embedding_layers):
            embedding_outputs.append(embedding_layer(categorical_input))
        categorical_inputs = self.embedded_concatenation(embedding_outputs)
        # print(embedding_outputs[0].shape)
        # print(embedding_outputs[0].shape)
        # print(categorical_inputs.shape)
        trans_input = self.pre_concatenation([continuous_inputs, categorical_inputs])
        
        for transformer in self.transformers:
            trans_input = transformer(trans_input)
        mlp_input = self.flatten_transformer_output(trans_input)
        # print(categorical_inputs.shape)
        # --> MLP
        # print(mlp_input.shape)
        for mlp_layer in self.mlp_layers:
            mlp_input = mlp_layer(mlp_input)
        return self.output_layer(mlp_input)

In [13]:
with strategy.scope():
  model = FTTransformer(
        categories = [], # number of unique elements in each categorical feature
        num_continuous = 177,      # number of numerical features
        dim = 16,                # embedding/transformer dimension
        dim_out = 1,             # dimension of the model output
        depth = 6,  
        embedding_dim=256,             # number of transformer layers in the stack
        heads = 8,               # number of attention heads
        attn_dropout = 0.1,      # attention layer dropout in transformers
        ff_dropout = 0.1,        # feed-forward layer dropout in transformers
        mlp_hidden = [(1024, 'relu'),(256, 'relu'),(128, 'relu'),(64, 'relu'), (32, 'relu')] # mlp layer dimensions and activations
    )

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
pip install --quiet tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf 
import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [11]:
def configure_device():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()  # connect to tpu cluster
        strategy = tf.distribute.TPUStrategy(tpu) # get strategy for tpu
        print('Num of TPUs: ', strategy.num_replicas_in_sync)
        device='TPU'
    except: # otherwise detect GPUs
        tpu = None
        gpus = tf.config.list_logical_devices('GPU') # get logical gpus
        ngpu = len(gpus)
        if ngpu: # if number of GPUs are 0 then CPU
            strategy = tf.distribute.MirroredStrategy(gpus) # single-GPU or multi-GPU
            print("> Running on GPU", end=' | ')
            print("Num of GPUs: ", ngpu)
            device='GPU'
        else:
            print("> Running on CPU")
            strategy = tf.distribute.get_strategy() # connect to single gpu or cpu
            device='CPU'
    return strategy, device, tpu

In [10]:
strategy, device, tpu = configure_device()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-02-29 12:23:48.215905: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14604 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:18:00.0, compute capability: 7.0
2024-02-29 12:23:48.216702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14604 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2024-02-29 12:23:48.217239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14604 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2024-02-29 12:23:48.217799: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 14604 MB memory:  -> device: 3, name: Tesla V100-PCIE-16GB, pci bus id

> Running on GPU | Num of GPUs:  4
